In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from tpot import TPOTRegressor
from sklearn.metrics import make_scorer
import matplotlib.pyplot as plt
from featuresv2 import targets, metadata
from position_group_features import features
import json
import requests
%matplotlib inline

## Version 4 Model

In [2]:
data = pd.read_csv('outputs/model_training_data.csv')

In [3]:
data2023 = data[data['round.year'] != 2023]

In [4]:
round_matchups = [
    ('Sydney Swans','Melbourne'),
    ('Brisbane Lions','Carlton'),
    ('Gold Coast Suns','Richmond'),
    ('GWS Giants','Collingwood')]

In [5]:
round1 = [('Carlton', 'Richmond'),
('Collingwood','Sydney Swans'),
('Essendon','Hawthorn'),
('GWS Giants','North Melbourne'),
('Geelong Cats','St Kilda'),
('Gold Coast Suns','Adelaide Crows'),
('Melbourne','Western Bulldogs'),
('Port Adelaide','West Coast Eagles'),
('Fremantle','Brisbane Lions')]

In [6]:
def generate_test_train_split(data, features, target, test_size=0.1):
    X = data[features]
    y = data[target]

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    
    return {"x_train":x_train,"x_test":x_test,"y_train":y_train,"y_test":y_test}

In [7]:
def tpot_pipeline(data, features, target, size=20, scoring='r2'):
    data_ = generate_test_train_split(data, features, target)

    my_scorer = make_scorer(mean_squared_error, greater_is_better=False)

    pipeline_optimizer = TPOTRegressor(generations=5, population_size=size, verbosity=2, scoring=scoring)

    pipeline_optimizer.fit(data_['x_train'], data_['y_train'])
    
    print(pipeline_optimizer.score(data_['x_test'], data_['y_test']))
    
    return pipeline_optimizer

In [8]:
def return_stats_input(data, home_team, away_team, model_inputs, games=5):
    '''Take in player_stats and returns aggregated team stats for previous home/away team matchups'''
    
#     if 'team.name' not in model_inputs:
#         model_inputs.append('team.name')
        
    match_ups = data[(data['match.homeTeam.name']==home_team) & (data['match.awayTeam.name']==away_team)]
    weights = range(1,len(match_ups)+1)

    stats = match_ups[model_inputs+['match.homeTeam.name']].groupby('match.homeTeam.name').apply(lambda x: np.average(x, weights=weights, axis=0))
    
    return pd.DataFrame(dict(zip(model_inputs,stats[0])),index=[0])

In [9]:
def generate_margin_tear_off(model, round_matchups, team_stats, model_inputs):
    print('-------------------' + str(model.fitted_pipeline_[0]).replace('()','') + ' TEAR-OFF-------------------')
    for tup in round_matchups:
        predict = model.predict(return_stats_input(team_stats, tup[0], tup[1], model_inputs))[0]
        if predict < 0:
            winner = tup[1]
        else:
            winner = tup[0]
        print("------------------------------------------------")
        print("Home: " + tup[0])
        print("Away: " + tup[1])
        print("Winner: " + winner + "\t       " + "Margin: " + str(abs(round(predict))))

In [10]:
def generate_score_tear_off(home_model, away_model, round_matchups, team_stats, model_inputs):
   # print('-------------------' + str(model.fitted_pipeline_[0]).replace('()','') + ' TEAR-OFF-------------------')
    preds = []
    for tup in round_matchups:
        try:
            h_predict = home_model.predict(return_stats_input(team_stats, tup[0], tup[1], model_inputs))[0]
            a_predict = away_model.predict(return_stats_input(team_stats, tup[0], tup[1], model_inputs))[0]
            if h_predict < a_predict:
                winner = tup[1]
                margin = round(a_predict) - round(h_predict)
            else:
                winner = tup[0]
                margin = round(h_predict) - round(a_predict)
            paddingh = 25 - len("Home: " + tup[0])
            paddinga = 25 - len("Away: " + tup[1])
            print("------------------------------------------------")
            print("Home: " + tup[0] + " "*paddingh + str(abs(round(h_predict))))
            print("Away: " + tup[1] + " "*paddinga + str(abs(round(a_predict))))
            print("Winner: " + winner + " by "+ str(margin) + " points")
            print("Total Score: " + str(round(h_predict) + round(a_predict)))
            preds.append((h_predict, a_predict, margin))
        except:
            print("No prior game")
            preds.append((0, 0, 0))
    return preds

In [11]:
home_model = tpot_pipeline(data, features, targets[0], size=10)

Optimization Progress:   0%|          | 0/60 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9564781414408232

Generation 2 - Current best internal CV score: 0.9568565963742085

Generation 3 - Current best internal CV score: 0.9568565963742085

Generation 4 - Current best internal CV score: 0.9568565963742085

Generation 5 - Current best internal CV score: 0.9568721904459769

Best pipeline: ElasticNetCV(RidgeCV(input_matrix), l1_ratio=0.6000000000000001, tol=1e-05)
0.9622897824040335


C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(


In [12]:
away_model = tpot_pipeline(data, features, targets[1], size=10)

Optimization Progress:   0%|          | 0/60 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9505056401580216

Generation 2 - Current best internal CV score: 0.9554718937655299

Generation 3 - Current best internal CV score: 0.9554718937655299

Generation 4 - Current best internal CV score: 0.9554718937655299

Generation 5 - Current best internal CV score: 0.9557734004960876

Best pipeline: RidgeCV(XGBRegressor(input_matrix, learning_rate=0.1, max_depth=1, min_child_weight=5, n_estimators=100, n_jobs=1, objective=reg:squarederror, subsample=0.2, verbosity=0))
0.9526817937744666


In [13]:
# margin_model = tpot_pipeline(data, features, targets[2], size=10)

In [14]:
#generate_score_tear_off(home_model, away_model, round1, data, features)

In [15]:
# print(return_stats_inputv3(data, home_team, away_team, features))
# generate_margin_tear_off(margin_model, round1, data, features)
# data[(data['match.homeTeam.name']==home_team) & (data['match.awayTeam.name']==away_team)][features].head()

In [16]:
fix_data_2024 = json.loads(requests.get('https://fixturedownload.com/feed/json/afl-2024').content)

fix_data_2023 = json.loads(requests.get('https://fixturedownload.com/feed/json/afl-2023').content)

fixtures2024 = pd.DataFrame(fix_data_2024)
games2024 = list(zip(fixtures2024.HomeTeam, fixtures2024.AwayTeam))

fixtures2023 = pd.DataFrame(fix_data_2023)
games2023 = list(zip(fixtures2023.HomeTeam, fixtures2023.AwayTeam))

predicted_season = generate_score_tear_off(home_model, away_model, games2024, data, features)

fixtures2024['predicted_home'] = [round(i[0]) for i in predicted_season]
fixtures2024['predicted_away'] = [round(i[1]) for i in predicted_season]

pred_winner = []
winner = []
for i, row in fixtures2024.iterrows():

    if row[7] > row[8]:
        winner.append(row[4])
    elif row[8] > row[7]:
        winner.append(row[5])
    else:
        winner.append('Draw')


    if row[9] > row[10]:
        pred_winner.append(row[4])
    elif row[10] > row[9]:
        pred_winner.append(row[5])
    else:
        pred_winner.append('Draw')

fixtures2024['predicted_winner'] = pred_winner
fixtures2024['winner'] = winner
fixtures2024['predicted_margin'] = fixtures2024['predicted_home'] - fixtures2024['predicted_away']
fixtures2024['actual_margin'] = fixtures2024['HomeTeamScore'] - fixtures2024['AwayTeamScore']
fixtures2024['correct'] = [1 if row[11]==row[12] else 0 for i,row in fixtures2024.iterrows()]
# fixtures2023['odds'] = [2.3 if abs(row[13]) < 40 else 2.6 for i,row in fixtures2023.iterrows()]


fixtures2024.to_csv('outputs/predictions_and_results_2024_.csv', index=False)

C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(


------------------------------------------------
Home: Sydney Swans       66
Away: Melbourne          70
Winner: Melbourne by 4 points
Total Score: 136
------------------------------------------------
Home: Brisbane Lions     93
Away: Carlton            67
Winner: Brisbane Lions by 26 points
Total Score: 160
------------------------------------------------
Home: Gold Coast Suns    77
Away: Richmond           91
Winner: Richmond by 14 points
Total Score: 168
------------------------------------------------
Home: GWS Giants         83
Away: Collingwood        69
Winner: GWS Giants by 14 points
Total Score: 152
------------------------------------------------
Home: Carlton            89
Away: Richmond           83
Winner: Carlton by 6 points
Total Score: 172
------------------------------------------------
Home: Collingwood        67
Away: Sydney Swans       46
Winner: Collingwood by 21 points
Total Score: 113
------------------------------------------------
Home: Essendon           99
Aw

C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(


------------------------------------------------
Home: Geelong Cats       98
Away: St Kilda           65
Winner: Geelong Cats by 33 points
Total Score: 163
------------------------------------------------
Home: Gold Coast Suns    100
Away: Adelaide Crows     78
Winner: Gold Coast Suns by 22 points
Total Score: 178
------------------------------------------------
Home: Melbourne          107
Away: Western Bulldogs   67
Winner: Melbourne by 40 points
Total Score: 174
------------------------------------------------
Home: Port Adelaide      109
Away: West Coast Eagles  52
Winner: Port Adelaide by 57 points
Total Score: 161
------------------------------------------------
Home: Fremantle          80
Away: Brisbane Lions     89
Winner: Brisbane Lions by 9 points
Total Score: 169


C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(


...

C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(


------------------------------------------------
Home: Carlton            68
Away: St Kilda           80
Winner: St Kilda by 12 points
Total Score: 148
------------------------------------------------
Home: Geelong Cats       127
Away: West Coast Eagles  60
Winner: Geelong Cats by 67 points
Total Score: 187
------------------------------------------------
Home: Hawthorn           77
Away: North Melbourne    65
Winner: Hawthorn by 12 points
Total Score: 142
------------------------------------------------
Home: Melbourne          77
Away: Collingwood        71
Winner: Melbourne by 6 points
Total Score: 148
------------------------------------------------
Home: Richmond           51
Away: Gold Coast Suns    61
Winner: Gold Coast Suns by 10 points
Total Score: 112


C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(


------------------------------------------------
Home: Sydney Swans       107
Away: Adelaide Crows     88
Winner: Sydney Swans by 19 points
Total Score: 195
------------------------------------------------
Home: Western Bulldogs   67
Away: GWS Giants         65
Winner: Western Bulldogs by 2 points
Total Score: 132
------------------------------------------------
Home: Fremantle          71
Away: Port Adelaide      78
Winner: Port Adelaide by 7 points
Total Score: 149


C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(


In [17]:
# backtest_data = pd.read_csv('outputs/predictions_and_results_2024.csv')
# backtest_data.columns

In [18]:
# profit_curve_no_geelong = []
# bankroll = 100
# stake = 0
# profit = 0
# for i, row in backtest_data.iterrows():
    
#     if (backtest_data['predicted_winner'][i] != 'Geelong Cats') and (backtest_data['predicted_winner'][i] != 'Richmond'):
#         stake = -1 * 0.1 * bankroll
#         profit = -stake * backtest_data['odds'][i] * backtest_data['win '][i]
        
#         bankroll+=stake+profit

#         profit_curve_no_geelong.append(bankroll)
#     else:
#         bankroll+=0

#         profit_curve_no_geelong.append(bankroll)

In [19]:
# profit_curve = []
# bankroll = 100
# stake = 0
# profit = 0
# for i, row in backtest_data.iterrows():
    
#     stake = -1 * 0.1 * bankroll
#     profit = -stake * backtest_data['odds'][i] * backtest_data['win '][i]

#     bankroll+=stake+profit

#     profit_curve.append(bankroll)

In [20]:
# plt.plot(profit_curve, color='b')
# plt.plot(profit_curve_no_geelong, color='r')